This file retrieves and stores MAL data - top 50 shows for now

In [1]:
## load libraries

import numpy as np
import pandas as pd

import selenium
from bs4 import BeautifulSoup
import requests

In [2]:
url = "https://myanimelist.net/topanime.php"
response = requests.get(url)
if response.status_code != 200:
    print('Error reading MAL Top 50')
else:
    soup_mal_top50 = BeautifulSoup(response.text, 'html5')

Load the top 50 table

In [3]:
soup_mal_top50_table = soup_mal_top50.find(class_="top-ranking-table")

Testing to retrieve data from the first item (defining functions in the meantime)

In [4]:
number1 = {}

def MAL_initEntry_top(soup):
    '''
    Takes the soup of an entry in the MAL top 50
    Returns a dictionay entry with Title and URL of the anime
    '''
    entry = {}
    entry['Title'] = soup.find(class_="hoverinfo_trigger").text
    entry['URL'] = soup.find(class_="hoverinfo_trigger").get('href')
    return entry

number1_soup = soup_mal_top50_table.find(class_="detail")
number1 = MAL_initEntry(number1_soup)

print(number1['Title'])
print(number1['URL'])

NameError: name 'MAL_initEntry' is not defined

Go to number1\['URL'\] and pull the soup

In [5]:
def MAL_retrieveEntry(entry):
    '''
    Takes a dictionary that has URL and Title of the MAL anime
    Returns a soup of the anine page in question
    '''
    response = requests.get(entry['URL'])
    if response.status_code != 200:
        print('Encountered ' + str(response.status_code) + ' error reading ' + entry['Title'])
        return -1
    else:
        return BeautifulSoup(response.text, 'html5')

soup_mal_number1 = MAL_retrieveEntry(number1)

KeyError: 'URL'

Retrieve more items from the first item

In [6]:
number1sidebar = soup_mal_number1.find(id='content')

NameError: name 'soup_mal_number1' is not defined

The below gives a list of all fields in the left column of the anime show page

In [7]:
number1sidebar.find_all(class_="dark_text")

NameError: name 'number1sidebar' is not defined

Cycle through the headers in the left column to get all entries. This does not take in multiple entries for a single column, only the first entry

In [8]:
def MAL_retrieveSidebar(anime_dict, soup):
    '''
    Returns anime_dict with additional raw data for the sidebar of the anime entry 
    '''
    sidebar = soup.find(id='content')
    headers = sidebar.find_all(class_="dark_text")
    for header in headers:
        column_name = header.text.strip()[:-1]
        #print(column_name)
        entry = header.next_sibling.strip()
        #print(entry)
        if(entry == ""):
            entry = header.findNext().text
        anime_dict[column_name] = entry
        #print('---')
    return anime_dict

number1 = MAL_retrieveSidebar(number1, soup_mal_number1)

NameError: name 'soup_mal_number1' is not defined

Aired, Duration, Members and Favorites needs post-processing

In [9]:
def MAL_ppDuration(duration_entry):
    '''
    Takes in the raw duration entry in the form (xx min.) (xx hr.) (per ep)
    Returns total number of minutes per episode
    '''
    duration_array = duration_entry.split()
    duration_norm = 0
    if 'min.' in duration_array:
        duration_norm += int(duration_array[duration_array.index('min.') - 1])
    if 'hr.' in duration_array:
        duration_norm += 60 * int(duration_array[duration_array.index('hr.') - 1])
    return duration_norm

In [10]:
def MAL_ppAired(anime_dict):
    '''
    Takes in the original anime sidebar dictionary
    Return the dictionary with 'Started' and 'Ended' columns added in Timestamp format
    '''
    def MAL_toDatetime(date_string):
        if '?' not in date_string:
            try:
                return pd.to_datetime(aired_array[0], format="%b %d, %Y")
            except:
                try:
                    return pd.to_datetime(aired_array[0], format="%b, %Y")
                except:
                    return pd.to_datetime(aired_array[0], format="%Y")
        else:
            return np.nan

    aired_array = anime_dict['Aired'].split(' to ')
    anime_dict['Started'] = MAL_toDatetime(aired_array[0])
    if len(aired_array) > 1:
        anime_dict['Ended'] = MAL_toDatetime(aired_array[1])
    return anime_dict

In [11]:
def remove_commas(input_str):
    return int(input_str.strip().replace(',',''))

def MAL_ppSidebar(sidebar_dict):
    '''
    Postprocessing of MAL sidebar
    Take in the unprocessed sidebar dictionary
    Return the processed sidebar dictionary
    '''

    if(sidebar_dict['Episodes'] == 'Unknown'):
        sidebar_dict['Episodes'] = np.nan
    else:
        sidebar_dict['Episodes'] = int(sidebar_dict['Episodes'])
    sidebar_dict['Duration'] = MAL_ppDuration(sidebar_dict['Duration'])
    sidebar_dict = MAL_ppAired(sidebar_dict)
    sidebar_dict['Members'] = remove_commas(sidebar_dict['Members'])
    sidebar_dict['Favorites'] = remove_commas(sidebar_dict['Favorites'])

    return sidebar_dict

In [12]:
number1 = MAL_ppSidebar(number1)

KeyError: 'Episodes'

Now retrieve the details from the topbar of the anime show page

In [13]:
def MAL_retrieveTopbar(anime_dict, soup):
    '''
    Retrieves the score and voters from the topbar of the anime entry
    Return the modified anime_dict
    '''

    topbar = soup.find(class_='anime-detail-header-stats')
    
    if(anime_dict['Score'] == 'N/A'):
        anime_dict['Score'] = np.nan 
        anime_dict['Voters'] = np.nan # no score means no one voted
    else:
        anime_dict['Score'] = float(topbar.find(class_='score-label').text)
        anime_dict['Voters'] = remove_commas(topbar.find(class_='score').get('data-user').split()[0])

    return anime_dict

In [14]:
number1 = MAL_retrieveTopbar(number1, soup_mal_number1)
print(number1['Score'])
print(number1['Voters'])

NameError: name 'soup_mal_number1' is not defined

To add related anime:

In [15]:
def MAL_retrieveRelated(anime_dict, soup):
    '''
    Retrieves related anime from the anime entry
    Returns the modified anime_dict
    '''
    related = soup.find(class_='anime_detail_related_anime')
    related_rows = related.find_all('tr')
    for item in related_rows:
        header = item.find('td').text.strip()[:-1]
        entry = item.find('td').find_next().text.strip()
        anime_dict[header] = entry
    
    return anime_dict

number1 = MAL_retrieveRelated(number1, soup_mal_number1)

NameError: name 'soup_mal_number1' is not defined

Test to see that all data is captured

In [16]:
number1

{}

For MPV, only the following fields are needed:

In [17]:
mal_columns = ['Title', 'Type', 'Episodes', 'Started', 'Duration', 'Score', 'Rating']

Now read the top 50 into a dictionary list

In [18]:
mal_anime_soup = soup_mal_top50_table.find_all(class_="detail")
mal_top50 = []
for anime_soup in mal_anime_soup:

    # proceses the entry in the top 50 page and gets the related page
    mal_entry = MAL_initEntry_top(anime_soup) 
    soup_mal_entry = MAL_retrieveEntry(mal_entry)
    
    # processes the anime page
    mal_entry = MAL_retrieveSidebar(mal_entry, soup_mal_entry)
    al_entry = MAL_ppSidebar(mal_entry)
    mal_entry = MAL_retrieveTopbar(mal_entry, soup_mal_entry)
    # not including this in MVP: mal_entry = MAL_retrieveRelated(mal_entry, soup_mal_entry)

    mal_top50.append(mal_entry)

Put the required columns into a dataframe

In [19]:
mal_df = pd.DataFrame(mal_top50)

In [20]:
mal_df

,Title,URL,English,Synonyms,Japanese,Type,Episodes,Status,Aired,Premiered,...,Duration,Rating,Score,Ranked,Popularity,Members,Favorites,Started,Ended,Voters
0,Fullmetal Alchemist: Brotherhood,https://myanimelist.net/anime/5114/Fullmetal_A...,Fullmetal Alchemist: Brotherhood,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...",鋼の錬金術師 FULLMETAL ALCHEMIST,TV,64,Finished Airing,"Apr 5, 2009 to Jul 4, 2010",Spring 2009,...,24,R - 17+ (violence & profanity),9.23,#1,#4,1737409,148802,2009-04-05,2009-04-05,1040597
1,Steins;Gate,https://myanimelist.net/anime/9253/Steins_Gate,Steins;Gate,NaN,STEINS;GATE,TV,24,Finished Airing,"Apr 6, 2011 to Sep 14, 2011",Spring 2011,...,24,PG-13 - Teens 13 or older,9.13,#2,#7,1419989,124382,2011-04-06,2011-04-06,767500
2,Gintama°,https://myanimelist.net/anime/28977/Gintama°,Gintama Season 4,Gintama' (2015),銀魂°,TV,51,Finished Airing,"Apr 8, 2015 to Mar 30, 2016",Spring 2015,...,24,PG-13 - Teens 13 or older,9.12,#3,#340,304801,8257,2015-04-08,2015-04-08,106198
3,Hunter x Hunter (2011),https://myanimelist.net/anime/11061/Hunter_x_H...,Hunter x Hunter,HxH (2011),HUNTER×HUNTER（ハンター×ハンター）,TV,148,Finished Airing,"Oct 2, 2011 to Sep 24, 2014",Fall 2011,...,23,PG-13 - Teens 13 or older,9.12,#4,#18,1157708,103982,2011-10-02,2011-10-02,637854
4,Ginga Eiyuu Densetsu,https://myanimelist.net/anime/820/Ginga_Eiyuu_...,Legend of the Galactic Heroes,"LoGH, LotGH, Gin'eiden, GinEiDen, Heldensagen ...",銀河英雄伝説,OVA,110,Finished Airing,"Jan 8, 1988 to Mar 17, 1997",NaN,...,26,R - 17+ (violence & profanity),9.11,#5,#624,186350,11723,1988-01-08,1988-01-08,41798
5,Gintama',https://myanimelist.net/anime/9969/Gintama,Gintama Season 2,Gintama (2011),銀魂',TV,51,Finished Airing,"Apr 4, 2011 to Mar 26, 2012",Spring 2011,...,24,PG-13 - Teens 13 or older,9.10,#6,#347,295007,5410,2011-04-04,2011-04-04,122194
6,Shingeki no Kyojin Season 3 Part 2,https://myanimelist.net/anime/38524/Shingeki_n...,Attack on Titan Season 3 Part 2,NaN,進撃の巨人 Season3 Part.2,TV,10,Finished Airing,"Apr 29, 2019 to Jul 1, 2019",Spring 2019,...,23,R - 17+ (violence & profanity),9.08,#7,#135,548141,18328,2019-04-29,2019-04-29,316489
7,Kimi no Na wa.,https://myanimelist.net/anime/32281/Kimi_no_Na_wa,Your Name.,NaN,君の名は。,Movie,1,Finished Airing,"Aug 26, 2016",NaN,...,106,PG-13 - Teens 13 or older,9.07,#8,#14,1251448,57069,2016-08-26,NaT,815621
8,Gintama': Enchousen,https://myanimelist.net/anime/15417/Gintama__E...,Gintama: Enchousen,"Gintama' (2012), Gintama' Overdrive, Kintama",銀魂' 延長戦,TV,13,Finished Airing,"Oct 4, 2012 to Mar 28, 2013",Fall 2012,...,24,PG-13 - Teens 13 or older,9.05,#9,#663,177118,1989,2012-10-04,2012-10-04,87213
9,3-gatsu no Lion 2nd Season,https://myanimelist.net/anime/35180/3-gatsu_no...,March Comes In Like A Lion 2nd Season,Sangatsu no Lion Second Season,3月のライオン 第2シリーズ,TV,22,Finished Airing,"Oct 14, 2017 to Mar 31, 2018",Fall 2017,...,25,PG-13 - Teens 13 or older,9.03,#10,#615,189687,7441,2017-10-14,2017-10-14,85457


Save it!

In [21]:
mal_df.to_pickle('mal_top50_mvp.pkl')

Now let's pull data from the alphabetical list

In [22]:
url = "https://myanimelist.net/anime.php?letter=A"
response = requests.get(url)
if response.status_code != 200:
    print('Error reading MAL First 50 starting with A')
else:
    soup_mal_first50A = BeautifulSoup(response.text, 'html5')

In [23]:
soup_mal_firstA = soup_mal_first50A.find('a', class_="hoverinfo_trigger")
soup_mal_firstA

<a class="hoverinfo_trigger" href="https://myanimelist.net/anime/40628/A_Brightening_Life" id="sarea40628" rel="#sinfo40628">
        <img alt="A Brightening Life" border="0" class="lazyload" data-src="https://cdn.myanimelist.net/r/50x70/images/anime/1565/103711.jpg?s=5932c0ea3750e0ca88955bd225c19305" data-srcset="https://cdn.myanimelist.net/r/50x70/images/anime/1565/103711.jpg?s=5932c0ea3750e0ca88955bd225c19305 1x, https://cdn.myanimelist.net/r/100x140/images/anime/1565/103711.jpg?s=69ef414fb936ecf19547e605f94adcef 2x" height="70" width="50"/>
      </a>

In [24]:
def MAL_initEntry_alpha(soup):
    '''
    Takes the soup of an entry in the MAL alphabetical list
    Returns a dictionay entry with Title and URL of the anime
    '''
    entry = {}
    entry['Title'] = soup.find("img").get('alt')
    entry['URL'] = soup.get('href')
    return entry

MAL_initEntry_alpha(soup_mal_firstA)

{'Title': 'A Brightening Life',
 'URL': 'https://myanimelist.net/anime/40628/A_Brightening_Life'}

In [25]:
soup_mal_first50A_list = soup_mal_first50A.find_all('a', class_="hoverinfo_trigger")
mal_firstA = []
for anime_soup in soup_mal_first50A_list:

    if(anime_soup.next_element.find('tion are valid entriesstrong') != -1):
        continue
    print(anime_soup)
    print("---")
    # proceses the entry in the top 50 page and gets the related page
    mal_entry = MAL_initEntry_alpha(anime_soup) 
    soup_mal_entry = MAL_retrieveEntry(mal_entry)
    
    # processes the anime page
    mal_entry = MAL_retrieveSidebar(mal_entry, soup_mal_entry)
    mal_entry = MAL_ppSidebar(mal_entry)
    mal_entry = MAL_retrieveTopbar(mal_entry, soup_mal_entry)
    # not including this in MVP: mal_entry = MAL_retrieveRelated(mal_entry, soup_mal_entry)

    mal_firstA.append(mal_entry)

" data-src="https://cdn.myanimelist.net/r/50x70/images/anime/12/82178.jpg?s=51179a8da8a0e405d2c52c230c58c148" data-srcset="https://cdn.myanimelist.net/r/50x70/images/anime/12/82178.jpg?s=51179a8da8a0e405d2c52c230c58c148 1x, https://cdn.myanimelist.net/r/100x140/images/anime/12/82178.jpg?s=387296867d22e66eb45cb93efb104d6d 2x" height="70" width="50"/>
      </a>
---
<a class="hoverinfo_trigger" href="https://myanimelist.net/anime/9433/A-jangcom" id="sarea9433" rel="#sinfo9433">
        <img alt="A-jang.com" border="0" class="lazyload" data-src="https://cdn.myanimelist.net/r/50x70/images/anime/2/26117.jpg?s=87959b7d249bf9ba67ec93e3396a5d8c" data-srcset="https://cdn.myanimelist.net/r/50x70/images/anime/2/26117.jpg?s=87959b7d249bf9ba67ec93e3396a5d8c 1x, https://cdn.myanimelist.net/r/100x140/images/anime/2/26117.jpg?s=c2e9a461357c0607991e38bbea732344 2x" height="70" width="50"/>
      </a>
---
<a class="hoverinfo_trigger" href="https://myanimelist.net/anime/1295/A-Ko_The_Versus" id="sarea129

In [26]:
mal_df = pd.DataFrame(mal_firstA)

In [27]:
mal_df

,Title,URL,English,Japanese,Type,Episodes,Status,Aired,Producers,Licensors,...,Ranked,Popularity,Members,Favorites,Started,Voters,Ended,Synonyms,Premiered,Broadcast
0,A Brightening Life,https://myanimelist.net/anime/40628/A_Brighten...,A brightening life,A brightening life,Movie,1.0,Finished Airing,2010,"None found,","None found,",...,#10255,#15794,102,1,2010-01-01,NaN,NaT,NaN,NaN,NaN
1,A Christmas Song,https://myanimelist.net/anime/39086/A_Christma...,A Christmas Song,A Christmas Song,Music,1.0,Finished Airing,"Nov 29, 2012",Avex Entertainment,"None found,",...,#10256,#14072,187,0,2012-11-29,NaN,NaT,NaN,NaN,NaN
2,A Kite,https://myanimelist.net/anime/320/A_Kite,Kite,A KITE（カイト）,OVA,2.0,Finished Airing,"Feb 25, 1998 to Oct 25, 1998",Green Bunny,Media Blasters,...,N/A,#2201,42394,190,1998-02-25,22203.0,1998-02-25,NaN,NaN,NaN
3,A Log Day of Timbre,https://myanimelist.net/anime/38712/A_Log_Day_...,A Log Day of Timbre,A Log Day of Timbre,ONA,1.0,Finished Airing,"Feb 25, 2011","None found,","None found,",...,#9715,#11950,385,0,2011-02-25,216.0,NaT,NaN,NaN,NaN
4,A New Journey,https://myanimelist.net/anime/39057/A_New_Journey,NaN,A New Journey,ONA,1.0,Finished Airing,"Jan 18, 2019","None found,","None found,",...,#6135,#9195,1312,5,2019-01-18,736.0,NaT,Season 2019: A New Journey | League of Legends,NaN,NaN
5,A Piece of Phantasmagoria,https://myanimelist.net/anime/6094/A_Piece_of_...,NaN,a piece of PHANTASMAGORIA (ア・ピース・オブ ファンタスマゴリア),OVA,15.0,Finished Airing,1995,Tamura Shigeru Studio,"None found,",...,#7023,#7080,3184,19,1995-01-01,922.0,NaT,NaN,NaN,NaN
6,A Play,https://myanimelist.net/anime/29902/A_Play,NaN,A PLAY,Music,1.0,Finished Airing,2011,"None found,","None found,",...,#10015,#11338,497,0,2011-01-01,269.0,NaT,NaN,NaN,NaN
7,A Smart Experiment,https://myanimelist.net/anime/30214/A_Smart_Ex...,NaN,A smart experiment,Special,1.0,Finished Airing,2006,"None found,","None found,",...,#9871,#11868,396,1,2006-01-01,157.0,NaT,A Small Experiment,NaN,NaN
8,A Tang Qi Yu,https://myanimelist.net/anime/37180/A_Tang_Qi_Yu,Tea Pets,阿唐奇遇,Movie,1.0,Finished Airing,"Jul 21, 2017","None found,","None found,",...,#10257,#14209,178,0,2017-07-21,NaN,NaT,"Atang Qiyu, Atang Qi Yu",NaN,NaN
9,A Woman in a Fashion Building,https://myanimelist.net/anime/37452/A_Woman_in...,NaN,A WOMAN IN A FASHION BUILDING,Movie,1.0,Finished Airing,2015,"None found,","None found,",...,#9872,#13651,215,0,2015-01-01,124.0,NaT,NaN,NaN,NaN


In [28]:
mal_df.to_pickle('mal_first50a_mvp.pkl')